In [1]:
import pandas as pd


In [2]:
r=pd.read_csv(r'/content/drive/MyDrive/movie_data/ratings.csv')

In [3]:
r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [4]:
r['userId'].nunique()

610

In [7]:
max=r['movieId'].value_counts().idxmax()

m=pd.read_csv(r'/content/drive/MyDrive/movie_data/movies.csv')

max_= m[m['movieId']== max]['title'].values[0]

max_

'Forrest Gump (1994)'

In [10]:
m.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [14]:
t=pd.read_csv(r'/content/drive/MyDrive/movie_data/tags.csv')

matrix=m[m['title'] == 'Matrix, The (1999)']['movieId'].values[0]

matrix_=t[t['movieId']== matrix]['tag'].unique()
matrix_

array(['martial arts', 'sci-fi', 'alternate universe', 'philosophy',
       'post apocalyptic'], dtype=object)

In [16]:
mo=m[m['title'] =='Terminator 2: Judgment Day (1991)']['movieId'].values[0]

mr=r[r['movieId']== mo]
av=mr['rating'].mean()
av

3.970982142857143

In [28]:
gr=r.groupby('movieId')['rating'].agg(['count','mean']).reset_index()

gr.columns=['movieId','rating_count','rating_mean']

gr

,movieId,rating_count,rating_mean
0,1,215,3.920930
1,2,110,3.431818
2,3,52,3.259615
3,4,7,2.357143
4,5,49,3.071429
...,...,...,...
9719,193581,1,4.000000
9720,193583,1,3.500000
9721,193585,1,3.500000
9722,193587,1,3.500000


In [30]:
mer=pd.merge(m,gr,on='movieId')

fi=mer[mer['rating_count'] > 50]

#result=fi.sort_values('rating_count',ascending=False).head(5)
sci=fi[fi['genres'].str.contains('Sci-Fi')]
sor=sci.sort_values('rating_count',ascending=False)

result=sor.iloc[2]

result


,418
movieId,480
title,Jurassic Park (1993)
genres,Action|Adventure|Sci-Fi|Thriller
rating_count,238
rating_mean,3.75


In [24]:
mer

,movieId,title,genres,num_ratings,avg_raing
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215,3.920930
1,2,Jumanji (1995),Adventure|Children|Fantasy,110,3.431818
2,3,Grumpier Old Men (1995),Comedy|Romance,52,3.259615
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,7,2.357143
4,5,Father of the Bride Part II (1995),Comedy,49,3.071429
...,...,...,...,...,...
9719,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,1,4.000000
9720,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,1,3.500000
9721,193585,Flint (2017),Drama,1,3.500000
9722,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,1,3.500000


In [26]:
result=mer.sort_values('num_ratings').head(5)
result

,movieId,title,genres,num_ratings,avg_raing
9723,193609,Andrew Dice Clay: Dice Rules (1991),Comedy,1,4.0
3010,4032,"Everlasting Piece, An (2000)",Comedy,1,4.0
6653,57526,Untraceable (2008),Crime|Thriller,1,3.0
6652,57522,First Sunday (2008),Comedy|Crime,1,3.5
6650,57502,Cat Soup (Nekojiru-so) (2001),Adventure|Animation|Drama|Horror,1,4.0


In [37]:
import requests
import numpy as np
from bs4 import BeautifulSoup

def scrapper(imdbId):
  id = str(int(imdbId))
  n_zeroes= 7 - len(id)
  new_id = "0"*n_zeroes + id
  URL = f"https://www.imdb.com/title/tt{new_id}"
  rh=  {'content-Type':'text/html; charset=UTF-8',
       'User-Agent':'Mozila/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0',
        'Accept-Encoding' : 'gzip, deflate, br'}

  re=requests.get(URL,headers=rh)
  soup= BeautifulSoup(re.text)
  imdb_r= soup.find('span', attrs={'itemprop': 'ratingValue'})
  return imdb_r.text if imdb_r else np.nan




In [38]:
l=pd.read_csv(r'/content/drive/MyDrive/movie_data/links.csv')

mm=pd.merge(fi,l,on='movieId')

mm['imdb_rating']=mm['imdbId'].apply(scrapper)

In [40]:
mm

,movieId,title,genres,rating_count,rating_mean,imdbId,tmdbId,imdb_rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215,3.920930,114709,862.0,NaN
1,2,Jumanji (1995),Adventure|Children|Fantasy,110,3.431818,113497,8844.0,NaN
2,3,Grumpier Old Men (1995),Comedy|Romance,52,3.259615,113228,15602.0,NaN
3,6,Heat (1995),Action|Crime|Thriller,102,3.946078,113277,949.0,NaN
4,7,Sabrina (1995),Comedy|Romance,54,3.185185,114319,11860.0,NaN
...,...,...,...,...,...,...,...,...
431,106782,"Wolf of Wall Street, The (2013)",Comedy|Crime|Drama,54,3.916667,993846,106646.0,NaN
432,109374,"Grand Budapest Hotel, The (2014)",Comedy|Drama,52,3.778846,2278388,120467.0,NaN
433,109487,Interstellar (2014),Sci-Fi|IMAX,73,3.993151,816692,157336.0,NaN
434,112852,Guardians of the Galaxy (2014),Action|Adventure|Sci-Fi,59,4.050847,2015381,118340.0,NaN


In [41]:
mm=mm.fillna(0)
retult= mm.loc[mm['imdb_rating'].idxmax()]['movieId']

result

,418
movieId,480
title,Jurassic Park (1993)
genres,Action|Adventure|Sci-Fi|Thriller
rating_count,238
rating_mean,3.75


In [47]:
#scifi=fi[fi['genres'].str.contains('Sci-Fi')]

#scifim=pd.merge(scifi,l, on='movieId')

#scifim['imdb_rating']=scifim['imdbId'].apply(scrapper)

scifim=scifim.fillna(0)
result=scifim.loc[scifim['imdb_rating'].idxmax()]

s= result['movieId']

s

32

In [48]:
s

32